In [ ]:
import torch
import os
import requests
from safetensors.torch import load_file
from diffusers import FluxPipeline
from attention_processor import LocalFlexAttnProcessor, LocalDownsampleFlexAttnProcessor, init_local_mask_flex, init_local_downsample_mask_flex

In [ ]:
bfl_repo="black-forest-labs/FLUX.1-dev"
device = torch.device('cuda')
dtype = torch.bfloat16
pipe = FluxPipeline.from_pretrained(bfl_repo, torch_dtype=dtype).to(device)

In [ ]:
height = 1024
width = 1024
down_factor, window_size = 4, 8
# Supported Configurations:
# down_factor, window_size = 1, 8
# down_factor, window_size = 1, 16
# down_factor, window_size = 1, 32
# down_factor, window_size = 4, 16
# down_factor, window_size = 4, 8
if down_factor == 1:
    init_local_mask_flex(height // 16, width // 16, text_length=512, window_size=window_size, device=device)
    attn_processors = {}
    for k in pipe.transformer.attn_processors.keys():
        attn_processors[k] = LocalFlexAttnProcessor()
else:
    init_local_downsample_mask_flex(height // 16, width // 16, text_length=512, window_size=window_size, down_factor=down_factor, device=device)
    attn_processors = {}
    for k in pipe.transformer.attn_processors.keys():
        attn_processors[k] = LocalDownsampleFlexAttnProcessor(down_factor=down_factor).to(device, dtype)
pipe.transformer.set_attn_processor(attn_processors)

In [ ]:
if not os.path.exists('ckpt'):
    os.mkdir('ckpt')
if down_factor == 1:
    if not os.path.exists(f'ckpt/clear_local_{window_size}.safetensors'):
        print(f'Checkpoint not found. Downloading checkpoint to ckpt/clear_local_{window_size}.safetensors')
        response = requests.get(f"https://huggingface.co/Huage001/CLEAR/resolve/main/clear_local_{window_size}.safetensors")
        response.raise_for_status()
        with open(f'ckpt/clear_local_{window_size}.safetensors', 'wb') as f:
            f.write(response.content)
    state_dict = load_file(f'ckpt/clear_local_{window_size}.safetensors')
else:
    if not os.path.exists(f'ckpt/clear_local_{window_size}_down_{down_factor}.safetensors'):
        print(f'Checkpoint not found. Downloading checkpoint to ckpt/clear_local_{window_size}_down_{down_factor}.safetensors')
        response = requests.get(f"https://huggingface.co/Huage001/CLEAR/resolve/main/clear_local_{window_size}_down_{down_factor}.safetensors")
        response.raise_for_status()
        with open(f'ckpt/clear_local_{window_size}_down_{down_factor}.safetensors', 'wb') as f:
            f.write(response.content)
    state_dict = load_file(f'ckpt/clear_local_{window_size}_down_{down_factor}.safetensors')

missing_keys, unexpected_keys = pipe.transformer.load_state_dict(state_dict, strict=False)

missing_keys = list(filter(lambda p: ('.attn.to_q.' in p or 
                                      '.attn.to_k.' in p or 
                                      '.attn.to_v.' in p or 
                                      '.attn.to_out.' in p or 
                                      'spatial_weight' in p), missing_keys))

if len(missing_keys) != 0 or len(unexpected_keys) != 0:
    print(
        f"Loading attn weights from state_dict led to unexpected keys: {unexpected_keys}"
        f" and missing keys: {missing_keys}."
    )

In [ ]:
prompt = "A Mickey is eating a pie"
height = 1024
width = 1024
image = pipe(
    prompt,
    height=height,
    width=width,
    guidance_scale=3.5,
    num_inference_steps=20,
    max_sequence_length=512,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image